In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from base_layer_utils import BaseLayerDataRepo, BaseLayerResultsRepo, ModelName
from base_layer_utils import compute_layer1_oof
from base_layer_utils import SklearnBLE

#from fast_text_data import FastTextDataGenerator# fasttext_data_process 
#from tfidf_data import tfidf_data_process

import pandas as pd
import numpy as np
import time
import gc
from sklearn.metrics import roc_auc_score

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


train_ori = pd.read_pickle('/home/kai/data/shiyi/data/flight_data/train_ori_1M.pkl').head(500000)
testX_ori = pd.read_pickle('/home/kai/data/shiyi/data/flight_data/testX_ori_100k.pkl')

In [3]:
train = pd.read_pickle('/home/kai/data/shiyi/data/flight_data/train_1M.pkl').head(500000)
testX = pd.read_pickle('/home/kai/data/shiyi/data/flight_data/testX_100k.pkl')
testY = np.load('/home/kai/data/shiyi/data/flight_data/testY_100k.npy')

In [4]:
train.shape, testX.shape, testY.shape

((500000, 9), (100000, 8), (100000,))

In [4]:
# train_rare = train.copy()
# train_rare.loc[train_rare['Month'].value_counts()[train_rare['Month']].values < 800, 'Month'] = 'RARE_VALUE'
# train_rare.head(3)

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,label
0,RARE_VALUE,19,2,18,21,157,133,6,0
1,4,3,1,12,18,79,172,4,0
2,2,1,6,9,15,129,72,2,0


In [5]:
train.dtypes

Month            int64
DayofMonth       int64
DayOfWeek        int64
DepTime          int64
UniqueCarrier    int64
Origin           int64
Dest             int64
Distance         uint8
label            int64
dtype: object

# tfidf data

In [ ]:
# compatible_models = [ModelName.XGB]#[ModelName.LGB, ModelName.LOGREG]
# ModelName.LGB_PERLABEL, ModelName.NBLGB, ModelName.NBLGB_PERLABEL,
# ModelName.XGB, ModelName.XGB_PERLABEL, ModelName.NBXGB, ModelName.NBXGB_PERLABEL,
# ModelName.LOGREG, ModelName.LOGREG_PERLABEL, ModelName.NBLOGREG, 
# ModelName.NBLOGREG_PERLABEL,
# ModelName.LSVC, ModelName.LSVC_PERLABEL, ModelName.NBLSVC, ModelName.NBLSVC_PERLABEL]

bldr.add_tfidf_data(train_sentence=train['comment_text'], test_sentence=test['comment_text'], 
                    y_train=train[label_cols], label_cols=label_cols, 
                    compatible_models=tfidfdata_compatible_models, word_ngram=(1,1), word_max=30000)

In [6]:
categorical_cols = ["Month", "DayofMonth", "DayOfWeek", "DepTime", "UniqueCarrier", "Origin", "Dest", "Distance"]
label_cols = ['label']
feature_cols = list(set(train.columns) - set(label_cols))

X_train_ordinal = train[feature_cols]
y_train = train[label_cols]#.to_dict('list')
X_test_ordinal = testX[feature_cols]
print(type(X_train_ordinal), type(y_train))

from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X_train_ordinal)
X_train_one_hot = enc.transform(X_train_ordinal)
X_test_one_hot = enc.transform(X_test_ordinal)

print(X_train_ordinal.shape, y_train.shape)

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>
(500000, 8) (500000, 1)


In [49]:
bldr = BaseLayerDataRepo()

bldr.add_data('flight_data_ordinal', X_train_ordinal, X_test_ordinal, y_train, label_cols, [ModelName.LGB])
bldr.add_data('flight_data_one_hot', X_train_one_hot, X_test_one_hot, y_train, label_cols, [ModelName.LOGREG, ModelName.XGB])

print(bldr)

data_id: flight_data_one_hot  
	x_train: (500000, 692)	x_test: (100000, 692)
	y_train type: <class 'dict'>
	compatible_models: {<ModelName.XGB: 1>, <ModelName.LOGREG: 9>}
 data_id: flight_data_ordinal  
	x_train: (500000, 8)	x_test: (100000, 8)
	y_train type: <class 'dict'>
	compatible_models: {<ModelName.LGB: 5>}
 


# Start generating oof_train, oof_test,  layer1 estimater prediction and model data id list

In [50]:
# uncomment any model to add to the model pool

model_pool = {}

SEED = 2018 

from sklearn.linear_model import LogisticRegression
# ###################################  Logreg normal   ####################################
logreg_ble = SklearnBLE(LogisticRegression, nb=False, seed=SEED)
model_pool[ModelName.LOGREG] = logreg_ble
# ###################################  Logreg nb       ####################################
# nblogreg_params = {'C':0.25}
# nblogreg_ble = SklearnBLE(LogisticRegression, nb=True, seed=SEED, params=nblogreg_params)
# model_pool[ModelName.NBLOGREG] = nblogreg_ble
# ###################################  Logreg per label  ##################################
# logreg_params_per_label = {
#     'identity_hate': {'C': 0.25, 'class_weight': 'balanced', 'fit_intercept': True},
#     'insult': {'C': 0.25, 'class_weight': 'balanced', 'fit_intercept': True},
#     'obscene': {'C': 0.7, 'class_weight': 'balanced', 'fit_intercept': True},
#     'severe_toxic': {'C': 0.3,'class_weight': None, 'fit_intercept': True},
#     'threat': {'C': 0.05, 'class_weight': 'balanced', 'fit_intercept': True}, 
#     'toxic': {'C': 0.8, 'class_weight': 'balanced', 'fit_intercept': True}
# }
# logreg_per_label_ble = SklearnBLE(LogisticRegression, nb=False, seed=SEED, per_label_params=logreg_params_per_label)
# model_pool[ModelName.LOGREG_PERLABEL] = logreg_per_label_ble



# from sklearn.svm import LinearSVC
# ###################################  Logreg normal   ####################################
# lsvc_ble = SklearnBLE(LinearSVC, need_calibrated_classifier_cv=True)
# model_pool[ModelName.LSVC] = lsvc_ble
# ###################################  Logreg nb   ####################################
# nblsvc_params = {'C':0.02}
# nblsvc_ble = SklearnBLE(LinearSVC, nb=True, seed=SEED, params=nblogreg_params, need_calibrated_classifier_cv=True)
# model_pool[ModelName.NBLSVC] = nblsvc_ble


from xgboost import XGBClassifier
# ###################################  XGB normal   ####################################
xgb_params = {'n_jobs': 5}
xgb_ble = SklearnBLE(XGBClassifier, seed=SEED, params=xgb_params)# XGBoostBLE(params=xgb_params, nb=False, seed=SEED)
model_pool[ModelName.XGB] = xgb_ble
# ###################################  XGB nb   ####################################
# nbxgb_ble = SklearnBLE(XGBClassifier, nb=True, seed=SEED, params=xgb_params)
# model_pool[ModelName.NBXGB] = nbxgb_ble



from base_layer_utils import LightgbmBLE
lgb_params = {
    'bagging_fraction': 0.5,
    'bagging_freq': 1,
    'bagging_seed': SEED,
    'boosting': 'gbdt',
    'best_round': 100, # this will be extract from grid search result
    'feature_fraction': 1,
    'feature_fraction_seed': SEED,
    'learning_rate': 0.1,
    'max_depth': -1,
    'metric': 'auc',
    'min_data_in_leaf': 20,
    'num_leaves': 31,
    'num_threads': 8,
    'objective': 'binary',
    'scale_pos_weight': 1,
    'categorical_feature': categorical_cols,
    'verbose_eval': 20
}
lgb_ble = LightgbmBLE(params=lgb_params)
model_pool[ModelName.LGB] = lgb_ble

# from lightgbm import LGBMClassifier
###################################  LGB normal   ####################################
# lgb_params = {'n_jobs': 3,'n_estimators': 50}
# lgb_ble = SklearnBLE(LGBMClassifier, seed=SEED, params=lgb_params)
# model_pool[ModelName.LGB] = lgb_ble
# ###################################  LGB nb   ####################################
# nblgb_ble = SklearnBLE(LGBMClassifier, nb=True, seed=SEED, params=lgb_params)
# model_pool[ModelName.NBLGB] = nblgb_ble
# ###################################  LGB per label   ####################################
# lgb_params_per_label = {}
# lgb_params_per_label['toxic'] = {'n_jobs': 8, 'num_leaves': 61}
# lgb_params_per_label['severe_toxic'] = {'n_jobs': 8, 'num_leaves': 11}
# lgb_params_per_label['obscene'] = {'n_jobs': 8, 'num_leaves': 61}
# lgb_params_per_label['threat'] = {'n_jobs': 8, 'num_leaves': 11}
# lgb_params_per_label['insult'] = {'n_jobs': 8,'num_leaves': 11}
# lgb_params_per_label['identity_hate'] = {'n_jobs': 8, 'num_leaves': 61}
# nblgb_per_label_ble = SklearnBLE(LGBMClassifier, seed=SEED, per_label_params=lgb_params_per_label)
# model_pool[ModelName.NBLGB_PERLABEL] = nblgb_per_label_ble

In [47]:
list(model_pool)

[<ModelName.XGB: 1>, <ModelName.LGB: 5>, <ModelName.LOGREG: 9>]

In [51]:
layer1_est_preds, layer1_oof_train, layer1_oof_mean_test, model_data_id_list = compute_layer1_oof(bldr, model_pool, label_cols, nfolds=5, sfm_threshold=None)

Computing... label: label        model_data_id: ModelName.XGB_flight_data_one_hot
processing fold 0...
processing fold 1...
processing fold 2...
processing fold 3...
processing fold 4...
Computing... label: label        model_data_id: ModelName.LGB_flight_data_ordinal
processing fold 0...
No evaluation set, thus not possible to use early stopping. Please train with your best params.


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


[20]	training's auc: 0.739713
[40]	training's auc: 0.757236
[60]	training's auc: 0.768935
[80]	training's auc: 0.776477
[100]	training's auc: 0.7824
processing fold 1...
No evaluation set, thus not possible to use early stopping. Please train with your best params.


/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/lightgbm/basic.py:681: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


[20]	training's auc: 0.73986
[40]	training's auc: 0.757638
[60]	training's auc: 0.769366
[80]	training's auc: 0.777548
[100]	training's auc: 0.783509
processing fold 2...
No evaluation set, thus not possible to use early stopping. Please train with your best params.
[20]	training's auc: 0.739337
[40]	training's auc: 0.75793
[60]	training's auc: 0.770115
[80]	training's auc: 0.777818
[100]	training's auc: 0.783956
processing fold 3...
No evaluation set, thus not possible to use early stopping. Please train with your best params.
[20]	training's auc: 0.739948
[40]	training's auc: 0.757353
[60]	training's auc: 0.768232
[80]	training's auc: 0.775743
[100]	training's auc: 0.781806
processing fold 4...
No evaluation set, thus not possible to use early stopping. Please train with your best params.
[20]	training's auc: 0.738562
[40]	training's auc: 0.757525
[60]	training's auc: 0.768925
[80]	training's auc: 0.776744
[100]	training's auc: 0.782466
No evaluation set, thus not possible to use ear

## submit the layer 1 estimator predictions. If they look fine, save them to BaseLayerResultsRepo

In [52]:
list(layer1_est_preds)

['ModelName.XGB_flight_data_one_hot',
 'ModelName.LOGREG_flight_data_one_hot',
 'ModelName.LGB_flight_data_ordinal']

In [53]:
from sklearn.metrics import roc_auc_score

print('LGB', roc_auc_score(testY, layer1_est_preds['ModelName.LGB_flight_data_ordinal']))
print('XGB', roc_auc_score(testY, layer1_est_preds['ModelName.XGB_flight_data_one_hot']))
print('LOGREG', roc_auc_score(testY, layer1_est_preds['ModelName.LOGREG_flight_data_one_hot']))

LGB 0.7383601269295124
XGB 0.7152229827376881
LOGREG 0.7232898472070477


In [81]:
avg = (layer1_est_preds['ModelName.LGB_flight_data_ordinal'] + 
layer1_est_preds['ModelName.LOGREG_flight_data_one_hot'] + 
layer1_est_preds['ModelName.XGB_flight_data_one_hot']) / 3

roc_auc_score(testY, avg)

In [83]:
weighted_avg = (0.7*layer1_est_preds['ModelName.LGB_flight_data_ordinal'] + 
0.2*layer1_est_preds['ModelName.LOGREG_flight_data_one_hot'] + 
0.1*layer1_est_preds['ModelName.XGB_flight_data_one_hot']) / 3

roc_auc_score(testY, weighted_avg)

0.737480398808783

### generate files to submit from layer 1 estimators

In [22]:
def write_predictions_to_file(base_layer_est_preds):
    for key in base_layer_est_preds:
        submission = pd.read_csv(PATH + 'sample_submission.csv')#.head(1000)
        submission[label_cols] = base_layer_est_preds[key]
        sub_id = int(time.time())
        print(sub_id)
        submission.to_csv('./BaseEstPreds/' + key + '_' + str(sub_id) + '.csv.gz', index=False, compression='gzip')

In [23]:
write_predictions_to_file(layer1_est_preds)

1522094161
1522094166
1522094171


# sanity check before save to BaseLayerResultsRepo

In [54]:
model_data_id_list # model_data we just computed in the layer 1

['ModelName.XGB_flight_data_one_hot',
 'ModelName.LGB_flight_data_ordinal',
 'ModelName.LOGREG_flight_data_one_hot']

In [57]:
label_cols

['label']

In [55]:
list(layer1_oof_train) # list keys (which are labels)

['label']

In [58]:
len(layer1_oof_train[label_cols[0]]) # number of models to stack (each model will predict one set of labels like: toxic, servere_toxic, etc..)

3

In [60]:
len(layer1_oof_train[label_cols[0]][0]) # examples in oof_train (meta features, x_train) (meta labels are in train[label])

500000

In [61]:
list(layer1_oof_mean_test) # all labels like toxic, servere_toxic, etc..)

['label']

In [59]:
len(layer1_oof_mean_test[label_cols[0]][0]) # examples in oof_test (will be used by meta model (after validation) to predict the final prediction)

100000

In [68]:
# read the doc string to BaseLayerResultsRepo to set params
base_layer_results_repo = BaseLayerResultsRepo(label_cols=label_cols, filepath='oof/', load_from_file=False)

In [69]:
base_layer_results_repo.add(layer1_oof_train, layer1_oof_mean_test, layer1_est_preds, model_data_id_list)

In [85]:
base_layer_results_repo.get_model_data_id_list()

['ModelName.XGB_flight_data_one_hot',
 'ModelName.LGB_flight_data_ordinal',
 'ModelName.LOGREG_flight_data_one_hot']

In [70]:
scores = base_layer_results_repo.show_scores()

0	ModelName.XGB_flight_data_one_hot
0	ModelName.LOGREG_flight_data_one_hot
0	ModelName.LGB_flight_data_ordinal


In [71]:
 # let's give some fake scores
base_layer_results_repo.add_score('ModelName.XGB_flight_data_one_hot', 0.7152)
base_layer_results_repo.add_score('ModelName.LOGREG_flight_data_one_hot', 0.7232)
base_layer_results_repo.add_score('ModelName.LGB_flight_data_ordinal', 0.7383)

ModelName.XGB_flight_data_one_hot already existed in the repo. score: 0 update to 0.7152
ModelName.LOGREG_flight_data_one_hot already existed in the repo. score: 0 update to 0.7232
ModelName.LGB_flight_data_ordinal already existed in the repo. score: 0 update to 0.7383


In [72]:
scores = base_layer_results_repo.show_scores()

0.7383	ModelName.LGB_flight_data_ordinal
0.7232	ModelName.LOGREG_flight_data_one_hot
0.7152	ModelName.XGB_flight_data_one_hot


In [73]:
base_layer_results_repo.save()

In [74]:
print('stack 1')

stack 1
